# Importamos las librerias necesarias# Torre de Hanoi - Algoritmo A*
# Introducción a la Inteligencia Artificial

## Problema de la Torre de Hanoi con 5 discos

### Descripción del problema
La Torre de Hanoi es un problema clásico que consiste en mover una torre de discos de diferentes tamaños de una varilla a otra, siguiendo estas reglas:
1. Solo se puede mover un disco a la vez
2. Un disco solo puede colocarse sobre otro disco más grande
3. Objetivo: mover todos los discos de la varilla izquierda (A) a la derecha (C)

### Estado inicial: Todos los discos en la varilla A
### Estado objetivo: Todos los discos en la varilla C

# Importamos las librerias necesarias

In [ ]:
import heapq
import time
import json
from typing import List, Tuple, Optional, Dict, Any
from datetime import datetime
import tracemalloc

# Definimos las clases necesarias

In [ ]:
class EstadoHanoi:
    """
    Representa un estado del problema de la Torre de Hanoi.
    Cada estado contiene las tres varillas con sus respectivos discos.
    """

    def __init__(self, varillas: List[List[int]]):
        self.varillas = [list(varilla) for varilla in varillas]  # Copia profunda

    def __eq__(self, other):
        return self.varillas == other.varillas

    def __hash__(self):
        return hash(tuple(tuple(varilla) for varilla in self.varillas))

    def __str__(self):
        return f"A:{self.varillas[0]} B:{self.varillas[1]} C:{self.varillas[2]}"

    def es_valido(self) -> bool:
        """Verifica si el estado actual es válido según las reglas de Hanoi"""
        for varilla in self.varillas:
            for i in range(len(varilla) - 1):
                if varilla[i] > varilla[i + 1]:  # Disco más grande sobre uno más pequeño
                    return False
        return True

    def obtener_movimientos_posibles(self) -> List['EstadoHanoi']:
        """Genera todos los estados posibles desde el estado actual"""
        movimientos = []

        for origen in range(3):
            if self.varillas[origen]:  # Si la varilla tiene discos
                disco = self.varillas[origen][-1]  # Disco superior

                for destino in range(3):
                    if origen != destino:
                        # Verificar si el movimiento es válido
                        if not self.varillas[destino] or self.varillas[destino][-1] > disco:
                            nuevo_estado = EstadoHanoi(self.varillas)
                            nuevo_estado.varillas[origen].pop()
                            nuevo_estado.varillas[destino].append(disco)
                            movimientos.append(nuevo_estado)

        return movimientos